<a href="https://colab.research.google.com/github/CNG85-HW/CNG_Voodeo/blob/main/CNG_Voodeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

def main():
    print("Upload your reference images and video.")
    reference_images, video_path = upload_files()

    # Define model path
    model_path = "/content/drive/MyDrive/Colab Notebooks/models/ssd_mobilenet_v2_coco_2018_03_29/saved_model"

    # Verify model path
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model not found at {model_path}. Verify the extraction step and path.")

    print("Loading model...")
    global model
    model = tf.saved_model.load(model_path)

    # Process reference images
    reference_tensors, reference_labels = preprocess_images(reference_images)

    # Detect objects in video
    detect_objects_in_video(video_path, reference_tensors, reference_labels)



In [ ]:
from google.colab import files
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from google.colab.patches import cv2_imshow

# Function to upload multiple images and one video
def upload_files():
    uploaded = files.upload()  # Drag and drop files here
    reference_images = {}
    video_path = None

    for file_name in uploaded.keys():
        if file_name.lower().endswith(('.jpg', '.png', '.jpeg')):
            object_name = file_name.rsplit('.', 1)[0]  # Use file name (without extension) as the label
            reference_images[object_name] = file_name
        elif file_name.lower().endswith(('.mp4', '.avi')):
            video_path = file_name

    if not reference_images or not video_path:
        raise ValueError("Please upload at least one reference image and one video file.")
    return reference_images, video_path

# Function to preprocess reference images
def preprocess_images(image_paths):
    processed_images = {}
    for label, path in image_paths.items():
        image = Image.open(path)
        image_np = np.array(image)
        input_tensor = tf.convert_to_tensor(image_np)
        processed_images[label] = input_tensor[tf.newaxis, ...]
    return processed_images

# Function to detect objects in the video
def detect_objects_in_video(video_path, reference_images):
    # Preprocess all reference images
    processed_images = preprocess_images(reference_images)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    frame_number = 0
    detection_results = {label: [] for label in processed_images.keys()}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Prepare frame for model inference
        input_tensor = tf.convert_to_tensor(frame)
        input_tensor = input_tensor[tf.newaxis, ...]

        # Run object detection
        detections = model.signatures['serving_default'](input_tensor)

        # Extract detection results
        detection_scores = detections['detection_scores'].numpy()
        detection_boxes = detections['detection_boxes'].numpy()
        detection_classes = detections['detection_classes'].numpy()

        # Check each reference image for matches
        for label, reference_tensor in processed_images.items():
            if detection_scores[0][0] > 0.85:  # Adjust confidence threshold as needed
                print(f"Object '{label}' detected in frame {frame_number} with confidence {detection_scores[0][0]:.2f}")
                detection_results[label].append(frame_number)
                cv2_imshow(frame)

        frame_number += 1

    cap.release()

    # Print summary of detections
    for label, frames in detection_results.items():
        print(f"\nObject '{label}' detected in {len(frames)} frame(s): {frames}")

# Main function to run the script
def main():
    print("Upload your reference images and video.")
    reference_images, video_path = upload_files()

    # Load TensorFlow model
    model_path = "/content/ssd_mobilenet_v2_coco_2018_03_29/saved_model"
    print("Loading model...")
    global model
    model = tf.saved_model.load(model_path)

    # Detect objects in the video
    detect_objects_in_video(video_path, reference_images)

# Run the main function
if __name__ == "__main__":
    main()


In [1]:
pip install ipykernel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.5 MB/s eta 0:00:00
